In [17]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute, QuantumRegister, ClassicalRegister, Aer
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [18]:
backend = FakeMontreal()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(backend) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [19]:
def construct_tiling(circ_to_append, qbs_with_gates, num_qubit):
    """The circuit to append must already be measured"""
    assert len(qbs_with_gates) <= num_qubit
    idle_ind = list(set(range(num_qubit)) - set(qbs_with_gates))
    qr = QuantumRegister(num_qubit)
    cr = ClassicalRegister(num_qubit)
    qc = QuantumCircuit(qr, cr)
    qc.append(circ_to_append, qbs_with_gates, qbs_with_gates)
    qc.measure(idle_ind, idle_ind)
    return qc

In [20]:
qbs_with_gates = [0, 1, 2]
num_qubit = 5
circ_to_append = random_circuit(len(qbs_with_gates), 10, 2, measure=True)
qc = construct_tiling(circ_to_append, qbs_with_gates, num_qubit)

In [21]:
qc = transpile(qc, backend=backend_noisy, optimization_level=3)
qc.draw(fold=-1, idle_wires=False)

global phase: 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                ░          ┌─┐   
 q152_3 -> 4 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────┤M├───
              ┌────────────┐┌────┐  ┌──────────┐               ┌────┐  ┌───────┐          ┌────┐              ┌────┐    ┌─────────────┐     ┌────┐    ┌──────────────┐                                                                                                                          ┌────┐   ┌───────┐                                                                                               ┌────┐  ┌─────────┐            ░       ┌─┐└╥┘   
q152_2 -> 11 ─┤ Rz(-1.656) ├┤ √X ├──┤ Rz(-π/2) ├─────────■─────┤ √X ├──┤ Rz(π) ├──■───────┤ √X ├──────■───────┤ √X ├────┤ Rz(-1.8923) ├─────┤ √X ├────┤ Rz(-0.73077) ├───────────────────────────────────────────────────────────────────────────────────────────────────■────────────────■─────┤ √X ├───┤ Rz(π) ├────────────────────────────────────────────────────────────■────────────────────────────■─────┤ √X ├──┤ Rz(π/2) ├────────────░───────┤M├─╫────
             ┌┴────────────┤├────┤┌─┴──────────┴┐┌────┐  │     └────┘  └───────┘  │       └────┘      │       └────┘    └─────────────┘     └────┘    └──────────────┘        ┌────┐  ┌──────────┐┌────┐     ┌──────────────┐┌────┐┌──────────────┐┌────┐┌────────────┐  │                │     └────┘   └───────┘               ┌───┐     ┌───┐                              │                            │     └────┘  └─────────┘            ░ ┌─┐   └╥┘ ║    
q152_1 -> 13 ┤ Rz(0.79641) ├┤ √X ├┤ Rz(-0.7885) ├┤ √X ├──┼────────────────────────┼───────────────────┼─────────────────────────────────────────────────────────────────■─────┤ √X ├──┤ Rz(-π/2) ├┤ √X ├──■──┤ Rz(0.031977) ├┤ √X ├┤ Rz(-0.91448) ├┤ √X ├┤ Rz(1.6647) ├──┼────────────────┼──────────────────────────────────────┤ X ├──■──┤ X ├──────────────────────────────┼────────────────────────────┼────────────────────────────────────░─┤M├────╫──╫────
             └┬────────────┤├────┤└─┬──────────┬┘└────┘┌─┴─┐┌─────────┐         ┌─┴─┐┌─────────────┐┌─┴─┐┌─────────────┐     ┌────┐    ┌─────────────┐     ┌────┐     ┌─┴─┐┌──┴────┴─┐└──────────┘└────┘┌─┴─┐└┬────────────┬┘├────┤├─────────────┬┘├────┤├────────────┤┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐  ┌────┐ ┌─────────────┐└─┬─┘┌─┴─┐└─┬─┘┌─────────┐┌────┐┌─────────┐┌─┴─┐┌──────────────────────┐┌─┴─┐┌─────────┐   ┌────┐  ┌─────────┐ ░ └╥┘┌─┐ ║  ║    
q152_0 -> 14 ─┤ Rz(2.3062) ├┤ √X ├──┤ Rz(-π/2) ├───────┤ X ├┤ Rz(π/2) ├─────────┤ X ├┤ Rz(0.30609) ├┤ X ├┤ Rz(0.87057) ├─────┤ √X ├────┤ Rz(-2.7157) ├─────┤ √X ├─────┤ X ├┤ Rz(π/2) ├──────────────────┤ X ├─┤ Rz(1.6651) ├─┤ √X ├┤ Rz(-1.6705) ├─┤ √X ├┤ Rz(2.3813) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├──┤ √X ├─┤ Rz(-2.1494) ├──■──┤ X ├──■──┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ X ├┤ Rz(5.89323497961975) ├┤ X ├┤ Rz(π/2) ├───┤ √X ├──┤ Rz(π/2) ├─░──╫─┤M├─╫──╫────
              └────────────┘└────┘  └──────────┘       └───┘└─────────┘         └───┘└─────────────┘└───┘└─────────────┘     └────┘    └─────────────┘     └────┘     └───┘└─────────┘                  └───┘ └────────────┘ └────┘└─────────────┘ └────┘└────────────┘└───┘└──────────┘└───┘└──────────┘  └────┘ └─────────────┘     └───┘     └─────────┘└────┘└─────────┘└───┘└──────────────

# Tiling Data Generation

In [22]:
def construct_mbl_tiling_circuit(qbs_with_gates, total_circuit_size):
    num_spins = 4 # Number of spins. Must be even.
    assert num_spins <= total_circuit_size
    W = 0.8 * np.pi # Disorder strength up to np.pi
    theta = 0.05 * np.pi # Interaction strength up to np.pi
    steps = 6

    disorders = generate_disorder(num_spins, W, seed=0)
    random_mbl_circuit = construct_mbl_circuit(num_spins, disorders, theta, steps)
    qc = construct_tiling(random_mbl_circuit, qbs_with_gates, total_circuit_size)
    qc = transpile(qc, backend=backend_noisy, optimization_level=3)
    return qc

In [23]:
qc = construct_mbl_tiling_circuit([0, 1, 2, 3], 8)
qc.draw(fold=-1, idle_wires=False)

┌─┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 q183_6 -> 4 ───────────────────┤M├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                └╥┘                       ┌────┐ ┌────────────┐     ┌────┐    ┌─────────────┐                                                                    ┌────┐    ┌────────────┐     ┌────┐    ┌─────────────┐                                                                    ┌────┐    ┌────────────┐     ┌────┐    ┌─────────────┐                                                                    ┌────┐    ┌────────────┐     ┌────┐    ┌─────────────┐                                                                   ┌────┐    ┌────────────┐     ┌────┐    ┌─────────────┐                                                              ┌────┐┌────────────┐┌────┐┌─────────────┐                                              ░ ┌─┐         
 q183_0 -> 5 ────────────────────╫──────────────────■─────┤ √X ├─┤ Rz(2.9845) ├─────┤ √X ├────┤ Rz(0.24536) ├────────────────────────────────────────────────────────────■───────┤ √X ├────┤ Rz(2.9845) ├─────┤ √X ├────┤ Rz(0.24536) ├────────────────────────────────────────────────────────────■───────┤ √X ├────┤ Rz(2.9845) ├─────┤ √X ├────┤ Rz(0.24536) ├────────────────────────────────────────────────────────────■───────┤ √X ├────┤ Rz(2.9845) ├─────┤ √X ├────┤ Rz(0.24536) ├────────────────────────────────────────────────────────────■──────┤ √X ├────┤ Rz(2.9845) ├─────┤ √X ├────┤ Rz(0.24536) ├───────────────────────────────────────────────────────────■──┤ √X ├┤ Rz(2.9845) ├┤ √X ├┤ Rz(0.24536) ├──────────────────────────────────────────────░─┤M├─────────
              ┌─────────┐ ┌────┐ ║ ┌──────────┐   ┌─┴─┐   ├────┤ ├────────────┤     ├────┤    ├─────────────┤           ┌────┐    ┌─────────────┐┌────┐ ┌────────────┐ ┌─┴─┐     ├────┤    ├────────────┤     ├────┤    ├─────────────┤               ┌────┐┌─────────────┐┌────┐ ┌────────────┐ ┌─┴─┐     ├────┤    ├────────────┤     ├────┤    ├─────────────┤               ┌────┐┌─────────────┐┌────┐ ┌────────────┐ ┌─┴─┐     ├────┤    ├────────────┤     ├────┤    ├─────────────┤               ┌────┐┌─────────────┐┌────┐ ┌────────────┐ ┌─┴─┐    ├────┤    ├────────────┤     ├────┤    ├─────────────┤               ┌────┐┌─────────────┐┌────┐ ┌────────────┐┌─┴─┐├────┤├────────────┤├────┤└──┬────────┬─┘     ┌────┐┌────────────┐┌────┐┌────────────┐ ░ └╥┘┌─┐      
 q183_1 -> 8 ─┤ Rz(π/2) ├─┤ √X ├─╫─┤ Rz(-π/2) ├───┤ X ├───┤ √X ├─┤ Rz(1.7279) ├─────┤ √X ├────┤ Rz(-2.0548) ├──■────────┤ √X ├────┤ Rz(-1.6444) ├┤ √X ├─┤ Rz(3.0027) ├─┤ X ├─────┤ √X ├────┤ Rz(1.7279) ├─────┤ √X ├────┤ Rz(-2.0548) ├─

In [24]:
from qiskit.transpiler.exceptions import TranspilerError
from scipy.linalg import LinAlgError

def generate_data(qbs_with_gates, total_num_qubits, save_path: str, n_entries_per_file: int):
    entries = []

    for i in tqdm_notebook(range(n_entries_per_file)):
        success = False
        while not success:
            try:
                circuit = construct_mbl_tiling_circuit(qbs_with_gates, total_num_qubits)
                success = True
            except (LinAlgError, TranspilerError, np.linalg.LinAlgError) as e:
                print(f"Ran into an error:, {e}")

        job_ideal = execute(circuit, **run_config_ideal)
        job_noisy = execute(circuit, **run_config_noisy)

        counts_ideal = job_ideal.result().get_counts()
        counts_noisy = job_noisy.result().get_counts()

        ideal_exp_val = cal_z_exp(counts_ideal)
        noisy_exp_val = cal_z_exp(counts_noisy)

        graph_data = circuit_to_graph_data_json(
            circuit=circuit,
            properties=properties,
            use_qubit_features=True,
            use_gate_features=True,
        )

        entry = ExpValueEntry(
            circuit_graph=graph_data,
            observable=[],
            ideal_exp_value=ideal_exp_val.tolist(),
            noisy_exp_values=[noisy_exp_val.tolist()],
            circuit_depth=circuit.depth()
        )
        entries.append(entry.to_dict())

    path = f"{save_path}/q{qbs_with_gates[0]}-q{qbs_with_gates[-1]}_total{total_num_qubits}.json"
    with open(path, "w") as f:
        # pprint(entries)
        json.dump(entries, f)

In [25]:
qbs_with_gates = [0, 1, 2, 3]
total_num_qubits = 8
# circuit = construct_mbl_tiling_circuit(qbs_with_gates, total_num_qubits)
generate_data(qbs_with_gates, total_num_qubits, './data/tiling/mbd', 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

# Training

In [26]:
train_paths = [
    './data/tiling/q0-q3_total8.json',
    './data/tiling/q2-q5_total8.json',
    './data/tiling/q4-q7_total8.json'
]

val_paths = [
     f'./data/mbd_datasets/val/step_{i}.json' for i in range(10)
]

In [27]:
BATCH_SIZE = 32

train_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        train_paths,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        val_paths,
    ),
    batch_size=BATCH_SIZE,
    shuffle=False
)

for data in train_loader:
    print(data)
    break

DataBatch(x=[5904, 22], edge_index=[2, 12224], edge_attr=[6320, 3], y=[32, 1, 8], observable=[32, 0], circuit_depth=[32, 1], noisy_0=[32, 1, 8], batch=[5904], ptr=[33])


In [28]:
model = ExpValCircuitGraphModel(
    num_node_features=22,
    hidden_channels=15,
    exp_value_size=8
)
criterion = torch.nn.MSELoss()

optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer,
                              'min',
                              factor=0.1,
                              patience=15,
                              verbose=True,
                              min_lr=0.00001)

In [29]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
print(len(train_loader) * BATCH_SIZE, len(val_loader) * BATCH_SIZE)

13645
3008 2016


In [ ]:
class ExpValCircuitGraphModel(torch.nn.Module):
    def __init__(
            self,
            num_node_features: int,
            hidden_channels: int
    ):
        super().__init__()

        self.transformer1 = TransformerConv(
            num_node_features, hidden_channels,
            heads=3,
            dropout=0.1
        )
        self.pooling1 = ASAPooling(hidden_channels * 3, 0.5)

        self.transformer2 = TransformerConv(
            hidden_channels * 3, hidden_channels,
            heads=2,
            dropout=0.1
        )
        self.pooling2 = ASAPooling(hidden_channels * 2, 0.5)

        self.body_seq = torch.nn.Sequential(
            Linear(hidden_channels * 2 + 5, hidden_channels),
            torch.nn.Dropout(0.2),
            Linear(hidden_channels, 4)
        )

    def forward(self,
                exp_value, observable,
                circuit_depth, nodes,
                edge_index, batch):
        graph = self.transformer1(nodes, edge_index)
        graph, edge_index, _, batch, _ = self.pooling1(
            graph, edge_index, batch=batch
        )

        graph = self.transformer2(graph, edge_index)
        graph, edge_index, _, batch, _ = self.pooling2(
            graph, edge_index, batch=batch
        )

        graph = global_mean_pool(graph, batch)

        merge = torch.cat((
            graph,
            torch.squeeze(exp_value, 1),
            circuit_depth
        ), dim=1)

        return self.body_seq(merge)


model = ExpValCircuitGraphModel(
    num_node_features=22,
    hidden_channels=15
)
criterion = torch.nn.MSELoss()

optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer,
                              'min',
                              factor=0.1,
                              patience=15,
                              verbose=True,
                              min_lr=0.00001)

min_valid_loss = np.inf

train_losses = []
val_losses = []

N_EPOCHS = 100

progress = tqdm_notebook(range(N_EPOCHS), desc='Model training', leave=True)
for epoch in progress:
    train_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        out = model(
            data.noisy_0,
            data.observable,
            data.circuit_depth,
            data.x,
            data.edge_index,
            data.batch
        )
        loss = criterion(out, torch.squeeze(data.y, 1))

        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    valid_loss = 0.0
    model.eval()
    for i, data in enumerate(val_loader):
        out = model(
            data.noisy_0,
            data.observable,
            data.circuit_depth,
            data.x,
            data.edge_index,
            data.batch)
        loss = criterion(out, torch.squeeze(data.y, 1))

        valid_loss += loss.item()

    scheduler.step(valid_loss)

    if epoch >= 1:
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(valid_loss / len(val_loader))

        progress.set_description(f"{round(train_losses[-1], 5)}, {round(val_losses[-1], 5)}")
        progress.refresh()

Model training:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_losses, label="train_loss")
plt.plot(val_losses, label="val_loss")
plt.yscale('log')

plt.legend()
plt.show()
model_path = './model/tiling/tiling_mbd.pth'

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
import pickle
to_save = {'train_losses': train_losses, 'val_losses': val_losses}
with open('.' + model_path.split('.')[1] + '.pk', 'wb') as handle:
    pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)